# Docker container as a development environment with Visual Studio Code

# IMPORTANT!
- VSC cannot be used efficiently using standard Dockerfiles
- Although this document works in very simple cases, it is better to refer to 20340 - Docker_Loccal_VSC...ipynb for more robust setup

Ref: 
- https://docs.microsoft.com/en-us/learn/modules/use-docker-container-dev-env-vs-code/
- https://code.visualstudio.com/docs/remote/containers-tutorial

You'll add configuration files to an existing project. These files will tell Visual Studio Code how to build an environment where the project will "just work." You'll use the Remote - Containers configuration to configure the runtime. You'll also automate configuration of a development environment that will work for anyone who has Docker and Visual Studio Code.

## Prerequisites
Installed locally:
- [Visual Studio Code](https://code.visualstudio.com/)
- [Git](https://git-scm.com/)
- Docker Desktop:
    - [Windows: Docker Desktop](https://www.docker.com/products/docker-desktop)
    - [Mac: Docker Desktop 2.0+.](https://hub.docker.com/editions/community/docker-ce-desktop-mac)

## Clone a repo

```
Ensure that Docker Desktop is installed and running on your machine.

1. Copy the URL for the sample repo to your clipboard.
https://github.com/MicrosoftDocs/mslearn-python-products
2. Open a new instance of Visual Studio Code.
3. Select the Clone Repository button in the sidebar, or press F1 to open the Command Palette and search for Git: Clone.
4. Paste in the URL from your clipboard.
5. Select a location on your disk where the project can be cloned to.
6. Select Open from the notification in Visual Studio Code.
7. Select "Yes, I trust the authors".

DO NOT install Python
```

## Install the Remote - Containers extension

```
1. Press Ctrl + Shift + X to open the Extensions viewlet, or click on the "Extensions" icon in the activity bar.
2. Type remote containers.
3. Locate the Remote - Containers extension and select the Install button.
```

The Remote - Containers extension lets you grab a dev container with the specific technology stack or dependencies already set up for you, open a project, and find that your code just works without downloading anything on your local machine. Remote - Containers works by connecting Visual Studio Code to a running container. Workspace files are mounted from the local file system, or they're copied or cloned into the container.

Visual Studio Code extensions are installed and run inside the container. There, they have full access to the tools, platform, and file system. To you, the developer, the experience is the same as if you had opened the project normally in Visual Studio Code.

You can seamlessly switch your entire development environment just by connecting to a different container. The extension handles all of the setup based on a few configuration files contained in a folder called **.devcontainer**.

https://docs.docker.com/storage/bind-mounts/

<img src="images/dev-env-2-architecture-containers.png" alt="" style="width: 800px;"/>

The remote container extension uses the files in the .devcontainer folder, namely devcontainer.json, and an optional Dockerfile or docker-compose.yml, to create your dev containers.

First your image is built from the supplied Docker file or image name. Then a container is created and started using some of the settings in the devcontainer.json. Finally your Visual Studio Code environment is installed and configured again according to settings in the devcontainer.json.

Once all of this is done, your local copy of Visual Studio Code connects to the Visual Studio Code Server running inside of your new dev container.

## Windows Only : Configure Docker

```
1. Right-click on the Docker task bar item, select Settings > Resources > File Sharing and add the location where you cloned the sample project in the previous step. This option is not available if you have enabled the WSL 2 back-end.
```

## Add a dev container


```
1. Open the Command Palette -> F1.
2. Type add dev container.
3. Select Remote-Containers: Add Development Container Configuration Files.
4. Select the following options:
Select a container configuration definition->Python 3
Python version->3 (default)
Install Node.js->none
5. Select OK.
```

## Inspect configuration files

```
1. Notice that a new folder called .devcontainer has been added to the project.
2. Expand that folder and notice that it contains both a Dockerfile and a devcontainer.json file.
```

## Open the project in a container

```
1. Open the Command Palette.
2. Type reopen in container.
3. Select Remote Containers: Reopen in Container from the list of available options.

The container will begin building. The initial build might take a few minutes because a new image has to be pulled down and built on your machine. After the container has been built the first time, later builds will be much faster.
```

## Inspect the container

```
1. Press Ctrl + ` to open the integrated terminal in Visual Studio Code if it is not already open.
2. Notice that the terminal probably looks different from your normal terminal.
3. Run the following command to ensure that Python is there:
python --version
```

## Install project dependencies

```
pip3 install --user -r requirements.txt
```

## Run the project

```
1. Enter the following command in the terminal to start the project:
python app.py
2. Open the project in a browser by navigating to http://127.0.0.1:5000.
```

## Customize container settings

```
Reopen the project locally

Before you change the container configuration, you need to reopen the project locally.
1. Press F1 to open the Command Palette.
2. Type locally and select Remote-Containers: Reopen Locally.
3. Open the devcontainer.json file in the .devcontainer folder if it's not open already.


Install Visual Studio Code extensions

The container comes with the Microsoft Python extension. The Python extension enables snippets, linting, and IntelliSense in Python files. But the index.html file in the templates folder is a Jinja template, and a different extension needs to be installed to get syntax highlighting in that file.

1. Open the Command Palette.
2. Type extension and select Extensions: Install Extensions.
3. In the extension explorer on the right, search for jinja.
4. Right-click the Jinja extension from wholroyd and select Add to devcontainer.json.
5. Notice that the extension ID has been added to the extensions option.
```

## Rebuild the new container

```
Rebuild the new container
1. Open the Command Palette.
2. Type rebuild and select Remote-Containers: Rebuild and Reopen in Container.
```

## Run the app

```
Run the app
1. Press Ctrl + ` to open the Visual Studio Code integrated terminal.
2. Run the app by using the following command:
python app.py
3. Notice that you didn't have to install any dependencies. The application simply runs.
```

## Software installation via Dockerfile

```
ARG VARIANT=3
FROM mcr.microsoft.com/vscode/devcontainers/python:${VARIANT}
RUN apt-get update && export DEBIAN_FRONTEND=noninteractive \
    && apt-get install -y traceroute

- The RUN command creates a new layer. Layers are how the container knows what has changed and what in the container needs to be updated when you rebuild it. You should try to keep related logic together in the same RUN command so that you don't create unnecessary layers.
- The \ denotes a line break at the end of a line. You need it for multiple-line commands.
- The && is how you add a command to the RUN line.
- The DEBIAN_FRONTEND export avoids warnings when you go on to work with your container. When you're adding other software, you might instead use other flags or parameters, such as -y.
- The -y ensures that apt-get doesn't prompt you to confirm that you want to finish the installation. These prompts would cause the container build to fail because nobody would be there to select Y or N.
```

## devcontainer.json

```
//devcontainer.json
{
  "name": "Node.js Sample",
  "dockerFile": "Dockerfile",

  "settings": {
    "terminal.integrated.shell.linux": "/bin/bash"
  },

  "extensions": ["dbaeumer.vscode-eslint"],

  "forwardPorts": [3000],

  "postCreateCommand": "yarn install",

  "remoteUser": "node"
}
```